# Explore ctapipe DL2 data

In [ ]:
import ctapipe
print(f"ctapipe version {ctapipe.__version__}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from ctapipe.instrument import SubarrayDescription
from ctapipe.io import TableLoader

In [ ]:
from tqdm.auto import tqdm

In [ ]:
def read_table(filename, tel_ids=None, stop=None):
    """
    Read data from a file and return it as a pandas DataFrame.

    Parameters
    ----------
    filename : str
        The path to the input file.
    tel_ids : list or None, optional
        A list of telescope IDs to include in the data. If None, all telescopes are included. Default is None.
    stop : int or None, optional
        The number of events to read from the file. If None, all events are read. Default is None.
    Returns
    -------
    pandas.DataFrame
        The data read from the file as a pandas DataFrame.
    """
    loader = TableLoader(
        input_url=filename,
        load_dl1_parameters=True,
        load_dl2=True,
        load_instrument=True,
        load_simulated=True,
        load_true_parameters=True,
    )

    data = loader.read_telescope_events(telescopes=tel_ids, stop=stop)
    data.remove_columns([col for col in data.colnames if len(data[col].shape) > 1])
    return data.to_pandas()


In [ ]:
dl2_proton_filename = "../data/proton.dl2.h5"
dl2_gamma_filename = "../data/gamma-diffuse.dl2.h5"

In [ ]:
subarray = SubarrayDescription.from_hdf(dl2_proton_filename)

In [ ]:
# limited due to github action limited memory. Change to None to get full statistics
n_events = 100000

protons = read_table(dl2_proton_filename, stop=n_events)
gammas = read_table(dl2_gamma_filename, stop=n_events)

In [ ]:
protons.describe()

In [ ]:
gammas.describe()

In [ ]:
columns = protons.columns
columns

In [ ]:
numerical_columns = [col for col in columns if hasattr(protons[col], 'dtype') and np.issubdtype(protons[col].dtype, np.number)]

ncol = 2
nrow = len(numerical_columns) // ncol + 1
fig, axes = plt.subplots(ncols=ncol, nrows=nrow, figsize=(20, 5*nrow))

nbins = 100
opt = dict(bins=nbins, histtype='step', density=True, lw=2)
columns_in_logscale = [col for col in numerical_columns if 'intensity' in col]
print(columns_in_logscale)


for i, col in tqdm(enumerate(numerical_columns), total=len(numerical_columns)):
    ax = axes[i//ncol, i%ncol]
    mask = np.isfinite(protons[col])
    # if col in columns_in_logscale and protons[mask][col].min() > 0 and gammas[col].min() > 0:
    #     opt['bins'] = np.logspace(np.log10(protons[mask][col].min()), np.log10(protons[mask][col].max()), nbins)
    ax.hist(protons[mask][col], label='protons', **opt)

    mask = np.isfinite(gammas[col])
    ax.hist(gammas[mask][col], label='gammas', **opt)
    
    opt['bins'] = nbins
    ax.set_title(col)
    ax.legend()

In [ ]:
opt_intensity = dict(bins=np.logspace(1.65, 4, 50), histtype='step', density=False, lw=2)
plt.figure(figsize=(10, 5))
plt.hist(gammas['hillas_intensity'], **opt_intensity, label='gammas');
plt.hist(protons['hillas_intensity'], **opt_intensity, label='protons');
plt.hist(gammas['HillasReconstructor_average_intensity'], **opt_intensity, label='gammas (average)')
plt.hist(protons['HillasReconstructor_average_intensity'], **opt_intensity, label='protons (average)')

plt.yscale('log')
plt.xscale('log')
xticks = [50, 100, 200, 500, 1000, 10000]
plt.gca().set_xticks(xticks)
plt.gca().set_xticklabels([str(x) for x in xticks])
plt.xlabel('Intensity')
plt.ylabel('Number of events')
plt.grid(which='both')
plt.legend()


In [ ]:
opt_energy = dict(bins=np.logspace(-3, 3, 50), histtype='step', density=False, lw=2)
plt.figure(figsize=(10, 5))
plt.hist(gammas['true_energy'], **opt_energy, label='gammas');
plt.hist(protons['true_energy'], **opt_energy, label='protons');


plt.yscale('log')
plt.xscale('log')
plt.xlabel(f"True Energy [TeV]")
plt.ylabel('Number of events')
plt.grid(which='both')
plt.legend()
